# LEFT JOIN with Pandas

### 렉처 중간에 예제 문제가 있습니다. 풀면서 따라와주세요!
**첨부한 쿼리는 [이곳](https://www.w3schools.com/sql/trysql.asp?filename=trysql_select_all)에서 동작합니다.**

In [1]:
import pandas as pd

# 데이터 읽기

In [2]:
orders = pd.read_csv('https://raw.githubusercontent.com/dataitgirls3/Data/master/Orders.csv')
orderDetails = pd.read_csv('https://raw.githubusercontent.com/dataitgirls3/Data/master/OrderDetails.csv')
products = pd.read_csv('https://raw.githubusercontent.com/dataitgirls3/Data/master/Products.csv')
customers = pd.read_csv('https://raw.githubusercontent.com/dataitgirls3/Data/master/Customers.csv')
categories = pd.read_csv('https://raw.githubusercontent.com/dataitgirls3/Data/master/Categories.csv')
suppliers = pd.read_csv('https://raw.githubusercontent.com/dataitgirls3/Data/master/Suppliers.csv')
shippers = pd.read_csv('https://raw.githubusercontent.com/dataitgirls3/Data/master/Shippers.csv')
employees = pd.read_csv('https://raw.githubusercontent.com/dataitgirls3/Data/master/Employees.csv')

# 데이터 확인하기

In [3]:
customers.head(2)

,CustomerID,CustomerName,ContactName,Address,City,PostalCode,Country
0,1,Alfreds Futterkiste,Maria Anders,Obere Str. 57,Berlin,12209,Germany
1,2,Ana Trujillo Emparedados y helados,Ana Trujillo,Avda. de la Constitución 2222,México D.F.,5021,Mexico


In [4]:
orders.head(2)

,OrderID,CustomerID,EmployeeID,OrderDate,ShipperID
0,10248,90,5,1996-07-04,3
1,10249,81,6,1996-07-05,1


In [7]:
orderDetails.head(2)

,OrderDetailID,OrderID,ProductID,Quantity
0,1,10248,11,12
1,2,10248,42,10


In [8]:
products.head(2)

,ProductID,ProductName,SupplierID,CategoryID,Unit,Price
0,1,Chais,1,1,10 boxes x 20 bags,18.0
1,2,Chang,1,1,24 - 12 oz bottles,19.0


## 고객 별 주문수를 알려주세요.

In [10]:
'''
SELECT c.CustomerID
     , COUNT(o.OrderID) as num_orders
FROM Customers AS c
     LEFT JOIN Orders AS o ON c.CustomerID = o.CustomerID
GROUP BY c.CustomerID
'''

'\nSELECT c.CustomerID\n     , COUNT(o.OrderID) as num_orders\nFROM Customers AS c\n     LEFT JOIN Orders AS o ON c.CustomerID = o.CustomerID\nGROUP BY c.CustomerID\n'

* 여기에서 왜 LEFT JOIN이 필요한가? -> 주문을 하나도 하지 않은 고객이 있을 수 있기 때문이다. Orders 테이블에는 존재하지 않지만, Customers 테이블에만 있는 고객들이 있기 때문이다.
* 왜 CustomerName이 아니라 CustomerID를 사용해 GROUP BY를 해야하는가? -> 동명이인이 있을 수 있기 때문이다. 이름과 다르게, CustomerID는 한 사람에 하나만 부여되는 ID입니다. 주민등록번호라고 생각해도 좋겠네요!

In [11]:
customers_order = customers.merge(orders, how='left', on='CustomerID')

In [12]:
# 1번 고객은 Orders 테이블에 정보가 없는가보군요. -> 주문을 한 번도 안 함.
customers_order.head(2)

,CustomerID,CustomerName,ContactName,Address,City,PostalCode,Country,OrderID,EmployeeID,OrderDate,ShipperID
0,1,Alfreds Futterkiste,Maria Anders,Obere Str. 57,Berlin,12209,Germany,NaN,NaN,NaN,NaN
1,2,Ana Trujillo Emparedados y helados,Ana Trujillo,Avda. de la Constitución 2222,México D.F.,5021,Mexico,10308.0,7.0,1996-09-18,3.0


### Q1. 1번 고객이 진짜 없는지 확인 해 볼까요?

In [14]:
# Answer

### Q2. 고객 중 동명이인이 있나요?

In [27]:
# Answer

### 다시 문제로 돌아옵시다.

In [24]:
# size()와 count()의 차이
# customers_num_orders = customers_order.groupby('CustomerID')['OrderID'].size()
customers_num_orders = customers_order.groupby('CustomerID')['OrderID'].count()

In [23]:
customers_num_orders.sort_values()

CustomerID
1      0
26     0
32     0
40     0
22     0
      ..
75     6
65     7
87     7
63     7
20    10
Name: OrderID, Length: 91, dtype: int64

## 1996년 10월 동안 고객 별 주문 수를 계산하세요.

### 두 쿼리 중 어떤 쿼리가 맞는 결과를 출력할까요?

In [28]:
'''
SELECT c.CustomerID
     , COUNT(o.OrderID)
FROM Customers AS c
     LEFT JOIN Orders AS o ON c.CustomerID = o.CustomerID 
                           AND o.OrderDate BETWEEN '1996-10-01' AND '1996-10-31'
GROUP BY c.CustomerID
'''

"\nSELECT c.CustomerID\n     , COUNT(o.OrderID)\nFROM Customers AS c\n     LEFT JOIN Orders AS o ON c.CustomerID = o.CustomerID \n                           AND o.OrderDate BETWEEN '1996-10-01' AND '1996-10-31'\nGROUP BY c.CustomerID\n"

In [29]:
'''
SELECT c.CustomerID
     , COUNT(o.OrderID)
FROM Customers AS c
     LEFT JOIN Orders AS o ON c.CustomerID = o.CustomerID 
WHERE o.OrderDate BETWEEN '1996-10-01' AND '1996-10-31'
GROUP BY c.CustomerID
'''

"\nSELECT c.CustomerID\n     , COUNT(o.OrderID)\nFROM Customers AS c\n     LEFT JOIN Orders AS o ON c.CustomerID = o.CustomerID \nWHERE o.OrderDate BETWEEN '1996-10-01' AND '1996-10-31'\nGROUP BY c.CustomerID\n"

### 10월 중 주문만 뽑기

In [53]:
orders_october = orders.set_index('OrderDate').loc['1996-10-01':'1996-10-31',:]
orders_october.head(2)

,OrderID,CustomerID,EmployeeID,ShipperID
OrderDate,,,,
1996-10-01,10318,38,8,2
1996-10-02,10319,80,7,3


In [54]:
orders_october = orders_october.reset_index()
orders_october.head(2)

,OrderDate,OrderID,CustomerID,EmployeeID,ShipperID
0,1996-10-01,10318,38,8,2
1,1996-10-02,10319,80,7,3


### Q3. 1996년 9월 주문만 뽑아서 orders_september에 저장해보세요.

In [55]:
# Answer

### 다시 원래 문제로 돌아와서,

In [61]:
customers_orders_october = customers.merge(orders_october, how='left', on='CustomerID')

In [62]:
customers_num_orders_october = customers_orders_october.groupby('CustomerID')['OrderID'].count()

In [64]:
customers_num_orders_october.sort_values()

CustomerID
1     0
63    0
62    0
61    0
59    0
     ..
25    2
51    2
87    2
38    2
39    2
Name: OrderID, Length: 91, dtype: int64

### Q4. CustomerID 39번 손님은 1996년 10월 중에 2번 주문을 했네요. 확인 해볼까요?

In [67]:
# Answer

## 고객 별 주문금액을 알려주세요.

### Step 1. 주문 별 금액 계산

In [71]:
orders_detail = orders.merge(orderDetails, how='inner', on='OrderID').merge(products, how='inner', on='ProductID')[['OrderID', 'CustomerID', 'OrderDate', 'OrderDetailID', 'Quantity', 'Price']]

In [77]:
orders_detail['total_price'] = orders_detail['Quantity'] * orders_detail['Price']

In [78]:
price_by_order = orders_detail.groupby(['OrderID', 'CustomerID', 'OrderDate'])['total_price'].sum().reset_index()

In [79]:
price_by_order.head(5)

,OrderID,CustomerID,OrderDate,total_price
0,10248,90,1996-07-04,566.00
1,10249,81,1996-07-05,2329.25
2,10250,34,1996-07-08,2267.25
3,10251,84,1996-07-08,839.50
4,10252,76,1996-07-09,4662.50


In [82]:
# 계산 결과가 의도대로 나왔는지 테스트
orders_detail[orders_detail['OrderID'] == 10248]['total_price'].sum()

566.0

### Step 2. 고객 데이터와 붙이기

In [84]:
customers_price_by_order = customers.merge(price_by_order, how='left', on='CustomerID')

In [85]:
customers_price_by_order.head(2)

,CustomerID,CustomerName,ContactName,Address,City,PostalCode,Country,OrderID,OrderDate,total_price
0,1,Alfreds Futterkiste,Maria Anders,Obere Str. 57,Berlin,12209,Germany,NaN,NaN,NaN
1,2,Ana Trujillo Emparedados y helados,Ana Trujillo,Avda. de la Constitución 2222,México D.F.,5021,Mexico,10308.0,1996-09-18,111.0


In [88]:
customers_price_by_order['total_price'] = customers_price_by_order['total_price'].fillna(0)

In [90]:
price_by_customers = customers_price_by_order.groupby('CustomerID')['total_price'].sum()

In [91]:
price_by_customers

CustomerID
1        0.00
2      111.00
3      504.00
4     1723.75
5     5406.90
       ...   
87    5872.50
88    3020.00
89    4416.00
90     566.00
91     573.75
Name: total_price, Length: 91, dtype: float64

### Q5. 89번 고객이 총 주문한 금액은 4416불입니다. 결과가 맞는지 확인하세요.

In [94]:
# Answer

## 1996년 10월 동안 고객 별 주문 금액을 계산하세요. 10월 중 주문을 한 번도 하지 않은 고객 목록을 주세요.

### Step 1. 10월 주문 별 금액 계산

In [100]:
price_by_order_october = price_by_order.set_index('OrderDate').loc['1996-10-01':'1996-10-31',:].reset_index()

In [103]:
customers_price_by_order_october = customers.merge(price_by_order_october, how='left', on='CustomerID')

### Step 2. 고객 데이터와 붙이기

In [105]:
# Answer

In [106]:
# 나머지를 스스로 풀어보세요!